In [12]:
#pandas version '2.2.2'
import pandas as pd
#Install xlrd >= 2.0.1

In [13]:
try:
    #google colab only - put data in a folder "/content/drive/MyDrive/capstone_data/"
    from google.colab import drive
    colab = 1
    print("Running in Google Colab, load data to \"/content/drive/MyDrive/capstone_data/\" to import it.")
    drive.mount('/content/drive')
except:
    colab = 0
    print("Load data to \"/capstone_data/\" to import it.")

Load data to "/capstone_data/" to import it.


In [14]:
if colab == 1:
    df_egen = pd.read_excel("/content/drive/MyDrive/capstone_data/annual_generation_state.xls", skiprows=1, header=0)
elif colab == 0:
    df_egen = pd.read_excel("capstone_data/annual_generation_state.xls", skiprows=1, header=0)

In [15]:
df_egen

,YEAR,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours)
0,1990,AK,Total Electric Power Industry,Total,5599506.0
1,1990,AK,Total Electric Power Industry,Coal,510573.0
2,1990,AK,Total Electric Power Industry,Hydroelectric Conventional,974521.0
3,1990,AK,Total Electric Power Industry,Natural Gas,3466261.0
4,1990,AK,Total Electric Power Industry,Petroleum,497116.0
...,...,...,...,...,...
62277,2023,WY,"Electric Generators, Electric Utilities",Coal,29830879.0
62278,2023,WY,"Electric Generators, Electric Utilities",Hydroelectric Conventional,908133.0
62279,2023,WY,"Electric Generators, Electric Utilities",Natural Gas,1430842.0
62280,2023,WY,"Electric Generators, Electric Utilities",Petroleum,43712.0


In [16]:
df_egen = df_egen[df_egen['YEAR'] >= 2000]
df_egen = df_egen[df_egen['YEAR'] <= 2019]

In [17]:
df_egen = df_egen[df_egen['TYPE OF PRODUCER'] == "Total Electric Power Industry"]

In [18]:
def categorize_energy(source):
    renewable_sources = [
        'Hydroelectric Conventional',
        'Geothermal',
        'Solar Thermal and Photovoltaic',
        'Wind',
        'Wood and Wood Derived Fuels',
        'Other Biomass',
        'Municipal Solid Waste',
        'Other Waste',
        'Other Renewable'
    ]
    if source == 'Total':
        return 'Total'
    elif source in renewable_sources:
        return 'Renewable'
    else:
        return 'Non-Renewable'

df_egen['Energy Category'] = df_egen['ENERGY SOURCE'].apply(categorize_energy)

df_categorized = df_egen.groupby(['YEAR', 'STATE', 'Energy Category'])['GENERATION (Megawatthours)'].sum().reset_index()

display(df_categorized.head())

,YEAR,STATE,Energy Category,GENERATION (Megawatthours)
0,2000,AK,Non-Renewable,5154706.0
1,2000,AK,Renewable,1001819.0
2,2000,AK,Total,6156525.0
3,2000,AL,Non-Renewable,114511544.0
4,2000,AL,Renewable,9893796.0


In [20]:
df_pivot = df_categorized.pivot_table(
    index=['YEAR', 'STATE'],
    columns='Energy Category',
    values='GENERATION (Megawatthours)'
).reset_index()

display(df_pivot.head())

Energy Category,YEAR,STATE,Non-Renewable,Renewable,Total
0,2000,AK,5154706.0,1001819.0,6156525.0
1,2000,AL,114511544.0,9893796.0,124405340.0
2,2000,AR,39911247.0,3964519.0,43875766.0
3,2000,AZ,80587778.0,8358799.0,88946577.0
4,2000,CA,147245036.0,60837447.0,208082483.0


In [21]:
df_pivot

Energy Category,YEAR,STATE,Non-Renewable,Renewable,Total
0,2000,AK,5154706.0,1001819.0,6156525.0
1,2000,AL,114511544.0,9893796.0,124405340.0
2,2000,AR,39911247.0,3964519.0,43875766.0
3,2000,AZ,80587778.0,8358799.0,88946577.0
4,2000,CA,147245036.0,60837447.0,208082483.0
...,...,...,...,...,...
1036,2019,VT,1913.0,2287885.0,2289798.0
1037,2019,WA,32182057.0,74281552.0,106463608.0
1038,2019,WI,56885038.0,5889260.0,62774299.0
1039,2019,WV,60588530.0,3337109.0,63925639.0


In [22]:
df_egen = df_pivot[df_pivot['STATE'] != "US-TOTAL"]
df_egen = df_egen[df_egen['STATE'] != "DC"]

In [23]:
df_egen

Energy Category,YEAR,STATE,Non-Renewable,Renewable,Total
0,2000,AK,5154706.0,1001819.0,6156525.0
1,2000,AL,114511544.0,9893796.0,124405340.0
2,2000,AR,39911247.0,3964519.0,43875766.0
3,2000,AZ,80587778.0,8358799.0,88946577.0
4,2000,CA,147245036.0,60837447.0,208082483.0
...,...,...,...,...,...
1036,2019,VT,1913.0,2287885.0,2289798.0
1037,2019,WA,32182057.0,74281552.0,106463608.0
1038,2019,WI,56885038.0,5889260.0,62774299.0
1039,2019,WV,60588530.0,3337109.0,63925639.0


In [24]:
if colab == 1:
    # Export to CSV (save to Google Drive)
    df_egen.to_csv('/content/drive/My Drive/capstone_data_cleaned/df_egen.csv', index=False)
elif colab == 0:
    df_egen.to_csv('capstone_data_cleaned/df_egen.csv', index=False)